In [1]:
# !pip install pymupdf reportlab

import fitz  # PyMuPDF
from reportlab.pdfgen import canvas
from reportlab.lib.colors import CMYKColor
from reportlab.pdfbase import pdfdoc

# 1. Read first page of PDF
input_pdf = "desert.pdf"
doc = fitz.open(input_pdf)
page = doc[0]

# Extract page dimensions
orig_rect = page.rect
print("Original size (points):", orig_rect.width, "x", orig_rect.height)


Original size (points): 896.0 x 896.0


In [2]:
# Define bleed (mm → points)
bleed_mm = 3
bleed_points = bleed_mm * 72 / 25.4  # 1 inch = 25.4mm, 72pt per inch

new_width = orig_rect.width + 2 * bleed_points
new_height = orig_rect.height + 2 * bleed_points
print("New size with bleed:", new_width, "x", new_height)


New size with bleed: 913.007874015748 x 913.007874015748


In [3]:
output_pdf = "with_cut_line.pdf"

# Make a new PDF
new_doc = fitz.open()
new_page = new_doc.new_page(width=new_width, height=new_height)

# Place original page in center (leaving bleed margins)
target_rect = fitz.Rect(bleed_points, bleed_points,
                        bleed_points + orig_rect.width,
                        bleed_points + orig_rect.height)

new_page.show_pdf_page(target_rect, doc, 0)


8

In [4]:
# Use ReportLab to add the spot color cut line
packet = output_pdf
c = canvas.Canvas(packet, pagesize=(new_width, new_height))

# Define spot color CutContour (100% magenta)
spot = CMYKColor(0, 1, 0, 0, spotName="CUT")
c.setStrokeColor(spot)
c.setLineWidth(0.25)

# Draw cut line at trim size (inside bleed area)
c.rect(bleed_points, bleed_points,
       orig_rect.width, orig_rect.height,
       stroke=1, fill=0)

c.showPage()
c.save()


In [11]:
# Example: Final page target (cm)
final_width_cm = 21   # A4 width
final_height_cm = 29.7
dpi = 300

# Convert to PDF points
final_width_points = (final_width_cm / 2.54) * 72
final_height_points = (final_height_cm / 2.54) * 72
print("Target size:", final_width_points, "x", final_height_points)

# Scale content proportionally
scale_x = final_width_points / new_width
scale_y = final_height_points / new_height
scale = min(scale_x, scale_y)

print("Scale factor:", scale)


Target size: 595.275590551181 x 841.8897637795275
Scale factor: 2.413796004583884
